In [1]:
from data_algebra.data_ops import *
import data_algebra.env
import data_algebra.yaml
import data_algebra.PostgreSQL




_, _1, _2, _get = [None, None, None, lambda x: x] # don't look unbound
data_algebra.yaml.fix_ordered_dict_yaml_rep()
data_algebra.env.push_onto_namespace_stack(locals())

db_model = data_algebra.PostgreSQL.PostgreSQLModel()

scale = 0.237

ops = TableDescription('d', 
                 ['subjectID',
                  'surveyCategory',
                  'assessmentTotal',
                  'irrelevantCol1',
                  'irrelevantCol2']) .\
    extend({'probability': '(assessmentTotal * scale).exp()'}) .\
    extend({'probability': 'probability/probability.sum()'},
           partition_by='subjectID') .\
    extend({'row_number':'_.row_number()'},
           partition_by=['subjectID'],
           order_by=['probability', 'surveyCategory'],
           reverse=['probability'])

print(ops.to_python(pretty=True))


TableDescription(
    table_name="d",
    column_names=[
        "subjectID",
        "surveyCategory",
        "assessmentTotal",
        "irrelevantCol1",
        "irrelevantCol2",
    ],
).extend({"probability": "(assessmentTotal * 0.237).exp()"}).extend(
    {"probability": "(probability / probability.sum())"}, partition_by=["subjectID"]
).extend(
    {"row_number": "_.row_number()"},
    partition_by=["subjectID"],
    order_by=["probability", "surveyCategory"],
    reverse=["probability"],
)



In [2]:
sql = ops.to_sql(db_model, pretty=True)
print(sql)

SELECT "probability",
       "surveyCategory",
       "irrelevantCol2",
       "subjectID",
       "irrelevantCol1",
       "assessmentTotal",
       ROW_NUMBER() OVER (PARTITION BY "subjectID"
                          ORDER BY "subjectID") AS "row_number"
FROM
  (SELECT "surveyCategory",
          "irrelevantCol2",
          "subjectID",
          "irrelevantCol1",
          "assessmentTotal", ("probability" / SUM("probability")) OVER (PARTITION BY "subjectID") AS "probability"
   FROM
     (SELECT "surveyCategory",
             "irrelevantCol2",
             "subjectID",
             "irrelevantCol1",
             "assessmentTotal",
             EXP(("assessmentTotal" * 0.237)) AS "probability"
      FROM
        (SELECT "surveyCategory",
                "irrelevantCol1",
                "assessmentTotal",
                "irrelevantCol2",
                "subjectID"
         FROM "d") "SQ_0") "SQ_1") "SQ_2"
